In [1]:
import csv
from My_Tool.train_1 import train_end
from fer import FER2013
import transforms
import torch
import os
import numpy as np
import parser
from torch.utils import data
from My_Tool.train_1 import train_end
import pandas as pd

C:\Users\karsa\AppData\Local\Temp\ipykernel_72828\1461258723.py:8: DeprecationWarning: The parser module is deprecated and will be removed in future versions of Python
  import parser


In [2]:
os.getcwd()

'D:\\DeepLearning\\Jinx\\Facial-Expression-Recognition\\My_Project'

In [3]:
cut_size = 44

In [4]:
# Data
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(44),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

transform_test = transforms.Compose([
    transforms.TenCrop(cut_size),
    transforms.Lambda(lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops])),
])

train_set = FER2013(split = 'Training', transform=transform_train)
train_loader = data.DataLoader(train_set, batch_size=64, shuffle=True, num_workers=1)

Public_Test = FER2013(split='PublicTest', transform=transform_test)
Public_Test_loader = data.DataLoader(Public_Test, batch_size=64, shuffle=False, num_workers=1)

Private_Test = FER2013(split='PrivateTest', transform=transform_test)
Private_Test_loader = data.DataLoader(Private_Test, batch_size=64, shuffle=False, num_workers=1)

==> Preparing data..


In [5]:
from d2l import torch as d2l
import torch
from torch import nn
from torch.nn import functional as F

In [6]:
class Residual(nn.Module):
    def __init__(self, input_channels, num_channels, use_1x1conv=False, strides=1):
        super().__init__()
        self.conv1 = nn.Conv2d(input_channels, num_channels, kernel_size=3, padding=1, stride=strides)
        self.conv2 = nn.Conv2d(num_channels, num_channels, kernel_size=3, padding=1)

        if use_1x1conv:
            self.conv3 = nn.Conv2d(input_channels, num_channels, kernel_size=1, stride=strides)
        else:
            self.conv3 = None
        self.bn1 = nn.BatchNorm2d(num_channels)
        self.bn2 = nn.BatchNorm2d(num_channels)
    def forward(self, X):
        Y = F.relu(self.bn1(self.conv1(X)))
        Y =self.bn2(self.conv2(Y))
        if self.conv3:
            X = self.conv3(X)
        Y += X
        return F.relu(Y)

In [7]:
b1 = nn.Sequential(nn.Conv2d(3 , 64, kernel_size=7, stride=2, padding=3),
                   nn.BatchNorm2d(64), nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

def resnet_block(input_channels, num_channels, num_residuals, first_block=False):
    blk = []
    for i in range(num_residuals):
        if i == 0 and not first_block:
            blk.append(Residual(input_channels, num_channels, use_1x1conv=True, strides=2))
        else:
            blk.append(Residual(num_channels, num_channels))
        return blk

b2 = nn.Sequential(*resnet_block(64, 64, 2, first_block=True))
b3 = nn.Sequential(*resnet_block(64, 128, 2))
b4 = nn.Sequential(*resnet_block(128, 256, 2))
b5 = nn.Sequential(*resnet_block(256, 512, 2))

net = nn.Sequential(b1, b2, b3, b4, b5,
                    nn.AdaptiveAvgPool2d((1,1)),
                    nn.Flatten(), nn.Linear(512, 7))

In [ ]:
train_end(net, train_loader, Public_Test_loader, num_epochs=100, lr= 0.05, device=d2l.try_gpu())

training on cuda:0
